In [18]:
%run COMPLEJO.ipynb
%run VECTOR_MATRIZ.ipynb

In [19]:
import unittest
import math
import matplotlib.pyplot as plt
import numpy as np

In [74]:
class Systems:
    CLASSIC = 0
    PROBABILISTIC = 1
    QUANTUM = 2
    
    INVALID_SYSTEM = "The system is not valid"
    INCORRECT_FORMAT = "The matriz doesn't match de system format"
    INVALID_VECTOR = "The vector is not valid"
    @staticmethod
    def verifySystem(matrix, systemType):
        ans = False
        if(systemType==Systems.CLASSIC):
            ans = Systems.verifyClassic(matrix)
        elif(systemType==Systems.PROBABILISTIC):
            ans = Systems.verifyProbabilistic(matrix)
        elif(systemType==Systems.QUANTUM):
            ans = Systems.verifyQuantum(matrix)
        else: 
            raise Exception(Systems.INVALID_SYSTEM)
        return ans
    
    @staticmethod
    def verifyClassic(matrix):
        isValid = True
        one = Complejo(1,0)
        zero = Complejo(0,0)
        columnSum = Complejo(0,0)
        j = 0
        while(j<matrix.columnas and isValid):
            i=0
            while(i<matrix.filas and isValid):
                isValid = isValid and (matrix[i,j]==one or matrix[i,j]==zero)
                columnSum+=matrix[i,j]
                i+=1
            isValid= isValid and columnSum==one
            j+=1
        return isValid
            
    @staticmethod
    def verificarColumnasEstocasticas(matrix):
        isValid = True
        one = Complejo(1,0)
        zero = Complejo(0,0)
        columnSum = Complejo(0,0)
        j = 0
        while(j<matrix.columnas and isValid):
            i=0
            while(i<matrix.filas and isValid):
                s = matrix[i,j].parteReal
                isValid = isValid and matrix[i,j].parteImaginaria==0 and s>=0 and s<=1
                columnSum+=matrix[i,j]
                i+=1
            isValid= isValid and columnSum==one
            columnSum=Complejo(0,0)
            j+=1
        
        return isValid
    
    @staticmethod
    def verificarFilasEstocasticas(matrix):
        isValid = True
        one = Complejo(1,0)
        zero = Complejo(0,0)
        columnSum = Complejo(0,0)
        i = 0
        while(i<matrix.filas and isValid):
            j=0
            while(j<matrix.columnas and isValid):
                s = matrix[i,j].parteReal
                isValid = isValid and matrix[i,j].parteImaginaria==0 and s>=0 and s<=1
                columnSum+=matrix[i,j]
                j+=1
            isValid= isValid and columnSum==one
            columnSum=Complejo(0,0)
            i+=1
        return isValid
    
    
    @staticmethod
    def verifyProbabilistic(matrix):
        return Systems.verificarFilasEstocasticas(matrix) and Systems.verificarColumnasEstocasticas(matrix)
    
    @staticmethod
    def verifyQuantum(matrix):
        return matriz.esUnitaria()
    
    
    @staticmethod
    def verifyVector(state0, systemType):
        ans= state0.columnas==1
        if(systemType==Systems.CLASSIC):
            ans = ans and Systems.verifyClassic(state0)
        elif(systemType==Systems.PROBABILISTIC):
            ans = ans and Systems.verificarColumnasEstocasticas(state0)
        elif(systemType==Systems.QUANTUM):
            prueba=Matriz(state0.filas,1)
            for i in range(state0.filas):
                prueba[i,0]=state0[i,0]*state0[i,0].conjugado()
            ans = ans and Systems.verificarColumnasEstocasticas(state0)
        else: 
            raise Exception(Systems.INVALID_VECTOR)
        return ans
    
    def __init__(self, systemType, matrix: Matriz, state0, clicks, statesA=None, statesB=None, assembled=False):
        self.state = [True, None, None]
        self.statesA=statesA
        self.statesB=statesB
        self.assembled=assembled
        self.type=None
        #[0]=isValid system, [1]=matriz^clicks, [2]=state0->final[t]
        if(not Systems.verifySystem(matrix, systemType) or matrix.filas==0 or matrix.filas!=matrix.columnas or 
           not Systems.verifyVector(state0, systemType) or matrix.filas!=state0.filas):
            self.state[0]=False
        else:
            self.state[1]=matrix**clicks
            self.state[2]=self.state[1]*state0
   
    def success(self):
        return self.state[0]==True and self.state[1]!=None and self.state[2]!=None
    
    
    @staticmethod
    def obtainType(matrix):
        ans = -1
        if(Systems.verifyClassic(matrix)):
            ans = Systems.CLASSIC
        elif(Systems.verifyProbabilistic(matrix)):
            ans = Systems.PROBABILISTIC
        elif(Systems.verifyQuantum(matrix)):
            ans = Systems.QUANTUM
        return ans
    
    
    @staticmethod
    def assembleSystems(systemA, viA, systemB, viB, t):
        assembled = systemA.productoTensor(systemB)
        statei = viA.productoTensor(viB)
        return Systems(Systems.obtainType(systemA), assembled, statei, t, viA.filas, viB.filas, True)
    

In [69]:
M = Matriz (3,3, [[0,1/6, 5/6], [1/3, 1/2, 1/6], [2/3, 1/3, 0]])
a = Systems(1, M, Matriz(3,1,[[1/5],[7/10],[1/10]]), 4)
print(a.state[2])

['0.36']
['0.33']
['0.31']


In [77]:
animo=Matriz(2,2, [[1/3, 2/3], [2/3, 1/3]])
posicion=Matriz(3,3, [[0,1/6, 5/6], [1/3, 1/2, 1/6], [2/3, 1/3,0]])
a=Systems.assembleSystems(animo, Matriz(3,1,[[0.01],[ 0.9], [0.09]]), posicion, Matriz(2,1, [[0.05],[0.95]]), 3)
print(a.state[2])

['0.10']
['0.18']
['0.23']
['0.10']
['0.17']
['0.22']
